In [ ]:
%run params.ipynb
import rules.example

for name, rules in [('example', rules.example.build_rules())]:
    PARAMS['save_file'] = True
    PARAMS['name'] = name
    PARAMS['graph'] = rules['graph']          
    PARAMS['rules'] = rules['rules']        
    
    %run "01. structural analysis.ipynb"
    %run "02. build basins.ipynb"
    %run "03. analyse basins.ipynb"

In [ ]:
import re
import rules.v01_oscillating as rul
from rules.enzymes import *

base_name = re.sub(".+\.", "", rul.__name__)

all_nodes = {Arp2_3, CP, Cdc42, Cofilin, Coronin, Cortactin, 
             Dia1, Dia2, Ena_Vasp, IRSp53, LIMK, PAK, PIP2, PKD, Profilin, 
             ROCK, Rac1, RhoA, SSH, Thymosin, WASP, WAVE}

rules_map = {}
orig_rules = rul.build_rules() 
rules_map[base_name] = orig_rules

# knockdown nodes
for node in all_nodes:
    graph = orig_rules["graph"].copy()
    rules = orig_rules['rules']
    
    graph.remove_edges_from([(node, target) for target in list(graph[node])])      
    rules = re.sub('1:\s*'+node+'\*\s*=.+', '', rules)    
    rules = rules + '1: '+node+'* = False \n'
    
    rules_map[base_name+"_del_"+node] = {"graph": graph, "rules": rules}


# multi-knockdowns
for nodes in [[Cdc42, Rac1], [Dia1, Dia2]]:
    graph = orig_rules["graph"].copy()    
    rules = orig_rules['rules']
    
    for node in nodes:
        graph.remove_edges_from([(node, target) for target in list(graph[node])])      
        rules = re.sub('1:\s*'+node+'\*\s*=.+', '', rules)    
        rules = rules + '1: '+node+'* = False \n'
              
    rules_map[base_name+"_del_"+"_".join(nodes)] = {"graph": graph, "rules": rules}
    
sorted(rules_map.keys())

# print(rules_map['v01_oscillating_del_Cdc42']['rules'])

In [ ]:
import re
import rules.v02_fixing_Arp2_3 as rul
from rules.enzymes import *

base_name = re.sub(".+\.", "", rul.__name__)

all_nodes = {Arp2_3, CP, Cdc42, Cofilin, Coronin, Cortactin, 
             Dia1, Dia2, Ena_Vasp, IRSp53, LIMK, PAK, PIP2, PKD, Profilin, 
             ROCK, Rac1, RhoA, SSH, Thymosin, WASP, WAVE}

rules_map = {}
orig_rules = rul.build_rules() 
rules_map[base_name] = orig_rules

# knockdown nodes
for node in all_nodes:
    graph = orig_rules["graph"].copy()
    rules = orig_rules['rules']
    
    graph.remove_edges_from([(node, target) for target in list(graph[node])])      
    rules = re.sub('1:\s*'+node+'\*\s*=.+', '', rules)    
    rules = rules + '1: '+node+'* = False \n'
    
    rules_map[base_name+"_del_"+node] = {"graph": graph, "rules": rules}


# multi-knockdowns
for nodes in [{Cdc42, Rac1}, [Dia1, Dia2]]:
    graph = orig_rules["graph"].copy()    
    rules = orig_rules['rules']
    
    for node in nodes:
        graph.remove_edges_from([(node, target) for target in list(graph[node])])      
        rules = re.sub('1:\s*'+node+'\*\s*=.+', '', rules)    
        rules = rules + '1: '+node+'* = False \n'
              
    rules_map[base_name+"_del_"+"_".join(nodes)] = {"graph": graph, "rules": rules}
    
sorted(rules_map.keys())

# print(rules_map['v01_oscillating_del_Cdc42']['rules'])

In [ ]:
### run selected rules

%run params.ipynb

for name, rules in sorted(rules_map.items()):
    print("====== "+name)    
    PARAMS['name'] = name
    PARAMS['save_file'] = True
    PARAMS['graph'] = rules['graph']
    PARAMS['rules'] = rules['rules']
    PARAMS['chunk_size'] = 100
    PARAMS['chunks'] = 5
    PARAMS['report_basins'] = [[Actin_BR, Actin_ST]]
    %run "01. structural analysis.ipynb"
    %run "02. build basins.ipynb"
    %run "03. analyse basins.ipynb"

In [ ]:
# build summary for selected rules map
import pandas as pd
import numpy as np

def color_delta(s):
    base = s['base']    
    colors = [ 'red' if s > 0 else 'blue' if s < 0 else 'black' for s in np.sign(s - base)]    
    return [ 'color: {}'.format(c)  for c in colors]

cases = sorted(rules_map.keys())
rep = pd.read_csv("reports/"+cases[0]+"/weighted_activity.csv", index_col=0)
rep.columns = ["base"]
for case in cases[1:]:
    name = re.sub(base_name+"_del_", "", case)
    rep[name] = pd.read_csv("reports/"+case+"/weighted_activity.csv", index_col=0).ix[:, 0].rename(name)

rep['mean'] = rep.mean(axis=1)
rep.to_csv("reports/"+cases[0]+"_summary.csv")

styler = rep.style.apply(color_delta, axis=1)

with open('reports/'+cases[0]+'_summary.html', 'w') as fo:
    fo.write(styler.render())
    
styler